## Download *<u>Sanctioned Cases by local office and manager<u>* & *<u>Caseload Breakdown by Local Office<u>* for the following

In [5]:
import csv
import glob
import shutil
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

# Create a folder with format (11-November 2018) in the directory 
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')
dir_name = 'S://TANF Reports//!2019 TANF Reports//'
os.chdir(dir_name)


folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


# Move all .csv files in Downloads folder to S:drive(destination) folder
path = 'C://Users//nnagaraj//Downloads'
os.chdir(path)
source = os.listdir(path)
destination = os.path.join('S://TANF Reports//!2019 TANF Reports', folder_name )

for files in source:  
    if files.endswith(".csv"):
        shutil.move(files, destination)  
        

# Convert all .csv files in S:drive to Excel        
os.chdir(destination)        
for filename in os.listdir(destination):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(destination, filename))

        x = filename.replace("-", " ") + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title()
        writer = pd.ExcelWriter(str(x).title().replace(".Csv", "")+".xlsx")
        df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False, header = filename)
        writer.save()
        continue
    else:
        continue       

# Now that all csv have been converted to excel, delete all the csv files in the directory        
files = [file for file in glob.glob("*.csv") if not file.endswith("0.csv")]
for file in files:
    os.remove(file)
    
# Load Excel
sanc_count = pd.read_excel('sanctioned Cases By Local Office and Manager' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')
caseload_count = pd.read_excel('Caseload Breakdown By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')

# Sanction count in each local office
san_grp = sanc_count.groupby('local_office').agg({'sanction_count':'sum'}).reset_index()

# Total caseload (TEA/WorkPays) in each local office
caseload_grp = caseload_count.groupby('local_office').agg({'caseload_count':'sum'}).reset_index()

# Create a new dataframe df with columns: caseload_grp and san_grp
df = pd.merge(san_grp, caseload_grp, how = 'inner', on = 'local_office')

# Total wfs in each local office
total_wfs = caseload_count.groupby('local_office').agg({'name':'count'}).reset_index()

# Merge all columns to dataframe df2
df2 = pd.merge(df, total_wfs, how = 'inner', on ='local_office')

# Insert calculated columns to df2
df2['avg_case'] = df2['caseload_count'].div(df2['name'])

df2['per_san'] = (100*(df2['sanction_count'])).div(df2['caseload_count'])


# Save statistics in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx 
file_name = 'Caseload Breakdown by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='openpyxl')


if os.path.exists(file_name):
    book = openpyxl.load_workbook(file_name)
    writer.book = book
    
df2.to_excel(writer, sheet_name= str_prev_month + '_statistics', index=False)
writer.save()
writer.close()


# Creating variables for dataframe
max_caseload = df2.loc[(df2['caseload_count'].idxmax())].to_frame().T.reset_index()
min_caseload = df2.loc[(df2['caseload_count'].idxmin())].to_frame().T.reset_index()
max_caseload_wfs = df2.loc[(df2['avg_case'].idxmax())].to_frame().T.reset_index()
min_caseload_wfs = df2.loc[(df2['avg_case'].idxmin())].to_frame().T.reset_index()
max_per_san = df2.loc[(df2['per_san'].idxmax())].to_frame().T.reset_index()
min_per_san = df2.loc[(df2['per_san'].idxmin())].to_frame().T.reset_index()

"""
print('LO with maximum caseload count :\n{}'.format(max_caseload))
print('\n\nLO with minimum caseload count :\n{}'.format(min_caseload))
print('\n\nLO with maximum caseload, WFS Ratio :\n{}'.format(max_caseload_wfs)) 
print('\n\nLO with minimum caseload, WFS Ratio :\n{}'.format(min_caseload_wfs)) 
print('\n\nLO with maximum Sanction Percentage :\n{}'.format(max_per_san))
print('\n\nLO with minimum Sanction Percentage :\n{}'.format(min_per_san))
"""

# Local Office Column
lo_max_caseload = max_caseload.loc[0,'local_office']
lo_min_caseload = min_caseload.loc[0,'local_office']
lo_max_caseload_wfs = max_caseload_wfs.loc[0,'local_office']
lo_min_caseload_wfs = min_caseload_wfs.loc[0,'local_office']
lo_max_per_san = max_per_san.loc[0,'local_office']
lo_min_per_san = min_per_san.loc[0,'local_office']

# Number column
num_max_caseload = max_caseload.loc[0,'caseload_count']
num_min_caseload = min_caseload.loc[0,'caseload_count']
num_max_caseload_wfs = max_caseload_wfs.loc[0,'avg_case']
num_min_caseload_wfs = min_caseload_wfs.loc[0,'avg_case']
num_max_per_san = max_per_san.loc[0,'per_san']
num_min_per_san = min_per_san.loc[0,'per_san']

# DataFrame
stat_frame = pd.DataFrame({'Summary_Matrix': ['LO with Maximum Caseload',
                                              'LO with Minimum Caseload',
                                             'LO with Maximum Caseload, WFS Ratio',
                                             'LO with Minimum Caseload, WFS Ratio',
                                             'LO with Maximum Sanction Percentage',
                                             'LO with Minimum Sanction Percentage'],
                           'Local_Office':[lo_max_caseload,
                                           lo_min_caseload,
                                           lo_max_caseload_wfs,
                                           lo_min_caseload_wfs,
                                           lo_max_per_san,
                                           lo_min_per_san],
                           'Number': [num_max_caseload,
                                      num_min_caseload,
                                      num_max_caseload_wfs,
                                      num_min_caseload_wfs,
                                      num_max_per_san,
                                      num_min_per_san]},
                         columns = ['Summary_Matrix','Local_Office','Number'])
                             
            
# Save dataframe in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx
file_name = 'Caseload Breakdown by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='openpyxl')

if os.path.exists(file_name):
    book = openpyxl.load_workbook(file_name)
    writer.book = book
    
stat_frame.to_excel(writer, sheet_name = str_prev_month + '_summary', index=False)
writer.save()
writer.close()
